In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbr"
fs_method, fs_ratio = "f1w", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9107 | AUPR: 0.6242 | Acc: 95.19%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9169 | AUPR: 0.6415 | Acc: 95.26%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9201 | AUPR: 0.6522 | Acc: 95.15%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9131 | AUPR: 0.6306 | Acc: 95.24%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9185 | AUPR: 0.6452 | Acc: 95.22%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9204 | AUPR: 0.6500 | Acc: 95.21%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9204 | AUPR: 0.6500 | Acc: 95.21%
Internal test: AUC: 0.9203 | AUPR: 0.6726 | Acc: 95.11%
External test: AUC: 0.9203 | AUPR: 0.6527 | Acc: 95.06%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9143 | AUPR: 0.6057 | Acc: 94.78%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9167 | AUPR: 0.6203 | Acc: 95.00%
model(kernel=linear, C=1)
Valid:         AUC: 0.9164 | AUPR: 0.6204 | Acc: 94.99%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9167 | AUPR: 0.6203 | Acc: 95.00%
Internal test: AUC: 0.9182 | AUPR: 0.6451 | Acc: 94.84%
External test: AUC: 0.9168 | AUPR: 0.6053 | Acc: 94.56%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7606 | AUPR: 0.3630 | Acc: 93.77%
model(n_neighbors=5)
Valid:         AUC: 0.7870 | AUPR: 0.4229 | Acc: 94.29%
model(n_neighbors=7)
Valid:         AUC: 0.8064 | AUPR: 0.4603 | Acc: 94.49%
best model(n_neighbors=7)
Valid:         AUC: 0.8064 | AUPR: 0.4603 | Acc: 94.49%
Internal test: AUC: 0.8151 | AUPR: 0.4675 | Acc: 94.18%
External test: AUC: 0.8115 | AUPR: 0.4705 | Acc: 94.45%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8716 | AUPR: 0.4395 | Acc: 77.18%
best model()
Valid:         AUC: 0.8716 | AUPR: 0.4395 | Acc: 77.18%
Internal test: AUC: 0.8835 | AUPR: 0.4735 | Acc: 74.14%
External test: AUC: 0.8691 | AUPR: 0.4316 | Acc: 72.53%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9173 | AUPR: 0.6469 | Acc: 95.27%
model(C=10.0)
Valid:         AUC: 0.9176 | AUPR: 0.6473 | Acc: 95.24%
model(C=100.0)
Valid:         AUC: 0.9174 | AUPR: 0.6468 | Acc: 95.22%
best model(C=10.0)
Valid:         AUC: 0.9176 | AUPR: 0.6473 | Acc: 95.24%
Internal test: AUC: 0.9207 | AUPR: 0.6768 | Acc: 95.01%
External test: AUC: 0.9171 | AUPR: 0.6368 | Acc: 95.02%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8803 | AUPR: 0.5619 | Acc: 85.00%
model(criterion=log_loss)
Valid:         AUC: 0.8806 | AUPR: 0.5689 | Acc: 85.58%
model(criterion=entropy)
Valid:         AUC: 0.8806 | AUPR: 0.5689 | Acc: 85.58%
best model(criterion=log_loss)
Valid:         AUC: 0.8806 | AUPR: 0.5689 | Acc: 85.58%
Internal test: AUC: 0.8718 | AUPR: 0.5844 | Acc: 85.47%
External test: AUC: 0.8802 | AUPR: 0.5799 | Acc: 84.95%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9147 | AUPR: 0.6512 | Acc: 89.65%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9157 | AUPR: 0.6522 | Acc: 89.85%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9163 | AUPR: 0.6518 | Acc: 89.76%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9156 | AUPR: 0.6546 | Acc: 89.81%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9165 | AUPR: 0.6538 | Acc: 89.80%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9163 | AUPR: 0.6547 | Acc: 89.78%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9156 | AUPR: 0.6546 | Acc: 89.81%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9165 | AUPR: 0.6538 | Acc: 89.80%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9163 | AUPR: 0.6547 | Acc: 89.78%
best model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9165 | AUPR: 0.6538 | Ac

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9210 | AUPR: 0.6467 | Acc: 95.18%
model(criterion=squared_error)
Valid:         AUC: 0.9211 | AUPR: 0.6467 | Acc: 95.19%
best model(criterion=squared_error)
Valid:         AUC: 0.9211 | AUPR: 0.6467 | Acc: 95.19%
Internal test: AUC: 0.9231 | AUPR: 0.6836 | Acc: 95.45%
External test: AUC: 0.9193 | AUPR: 0.6535 | Acc: 95.11%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9177 | AUPR: 0.6345 | Acc: 95.25%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9182 | AUPR: 0.6387 | Acc: 95.22%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9186 | AUPR: 0.6422 | Acc: 95.30%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9186 | AUPR: 0.6422 | Acc: 95.30%
Internal test: AUC: 0.9166 | AUPR: 0.6720 | Acc: 95.38%
External test: AUC: 0.9121 | AUPR: 0.6298 | Acc: 95.13%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)